In [1]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import torch
from glob import glob
from tqdm import tqdm

from ms_regnet.tools.io import read_tiff_stack, write_tiff_stack

In [2]:
from torch.utils.data.dataset import Dataset
from ms_regnet.constrain.reader_zoo import ReaderZoo
from ms_regnet.average_template.fusion import FusionTool
from ms_regnet.constrain import restore_raw_image_from_output
from ms_regnet.datasets import SimpleBrain

In [3]:
# average

    
    

file_list = [
   "/media/root/01a3fb01-7912-492d-9561-d36a4da2ffef/zht/data/MS-RegNet/result/bezier_atlas/1/181215_488_22-56-20/181215_488_22-56-20.tiff",
"/media/root/01a3fb01-7912-492d-9561-d36a4da2ffef/zht/data/MS-RegNet/result/bezier_atlas/1/180724_20180723O2_LEFT_488_40ET_10-25-50/180724_20180723O2_LEFT_488_40ET_10-25-50.tiff"
            ]
template = None
count = 0
fig = plt.figure(figsize=(100, 200))
constrain = {"simi": True,
            "tra": True,
            "outline": True,
            "hpf": True,
            "hole": True,
            "ctx": True,
            "cp": True,
            "cbx": True,
            "bs": True}
dataset = SimpleBrain(file_list, constrain)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

fusion_tool = FusionTool()


for data in loader:
    name = data["name"]
    data.pop("name")
    fusion_tool.fusion(data)

cur_template = fusion_tool.finish_fusion()

def write_data(data, name, basedir: str):
    if not os.path.exists(os.path.join(basedir, name)):
        os.mkdir(os.path.join(basedir, name))
    for k in data.keys():
        if k == 'simi':
            write_tiff_stack(restore_raw_image_from_output(data[k]), os.path.join(basedir, name, name+".tiff"))
        else:
            write_tiff_stack(restore_raw_image_from_output(data[k]), os.path.join(basedir, name, f"{name}_{k}.tiff"))

print(cur_template.keys())
# write_data(cur_template, "template", "/media/root/01a3fb01-7912-492d-9561-d36a4da2ffef/zht/data/MS-RegNet/result/bezier_atlas")

(320, 456, 528)
(145920, 528)
(320, 456, 528)
(145920, 528)


KeyboardInterrupt: 

<Figure size 7200x14400 with 0 Axes>

In [6]:
from torch.nn.functional import interpolate


file_list = glob("/media/root/01a3fb01-7912-492d-9561-d36a4da2ffef/zht/data/MS-RegNet/result/test/reg/*/")
file_list.sort()
template = None
count = 0
constrain = {"simi": True,
            "annotation": True}
dataset = SimpleBrain(file_list, constrain)
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)
from ms_regnet.tools.deform import grid_sample_nearest
space_dir = "/media/root/01a3fb01-7912-492d-9561-d36a4da2ffef/zht/data/MS-RegNet/result/bezier_atlas/1/"
data_list = []
for data in tqdm(loader):
    name = data["name"][0]        
    data = data["annotation"]["img_raw"]
    if "template" in name:
        data_list.append(data)
    else:
        space_file = os.path.join(space_dir, name[:-6], "space.pkl")
        import pickle
        with open(space_file, "rb") as f:
            spaces = pickle.load(f)
        for space in spaces:
            space = torch.tensor(space)
            space = space.permute(0, 4, 1, 2, 3)
            space = interpolate(space, scale_factor=2, mode='trilinear')
            space = space.permute(0, 2, 3, 4, 1)
            data = grid_sample_nearest(data, space) 
        data_list.append(data)

  0%|                                           | 0/8 [00:00<?, ?it/s]

(320, 456, 528)
(145920, 528)


/media/root/6701ae9d-6612-4271-8d50-522d7f72528b/anaconda3/envs/zht/lib/python3.8/site-packages/torch/nn/functional.py:3451: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
 12%|████                            | 1/8 [10:47<1:15:29, 647.02s/it]

(320, 456, 528)
(145920, 528)


 25%|████████                        | 2/8 [21:32<1:04:37, 646.20s/it]

...

 75%|████████████████████████        | 6/8 [1:04:32<21:29, 644.89s/it]

(320, 456, 528)
(145920, 528)


 88%|████████████████████████████    | 7/8 [1:17:14<11:23, 683.15s/it]

(320, 456, 528)
(145920, 528)


100%|████████████████████████████████| 8/8 [1:17:50<00:00, 583.80s/it]


In [13]:
from collections import Counter

# data_list = [i.numpy().squeeze() for i in data_list]
shape = data_list[0].shape
average_annotation = np.zeros_like(data_list[0], dtype=data_list[0].dtype)
for i in tqdm(range(shape[0])):
    for j in range(shape[1]):
        for k in range(shape[2]):
            counter = Counter([x[i, j, k] for x in data_list])
            max_id, maxn = data_list[-1][i, j, k], 3
            for key, value in counter.items():
                if value > maxn:
                    maxn, max_id =value, key
            average_annotation[i, j, k] = max_id            

100%|███████████████████████████████| 320/320 [05:31<00:00,  1.04s/it]


In [14]:
write_tiff_stack(average_annotation, "average_annotation.tiff")

import pickle
from ms_regnet.average_template.fusion import GrubbsOutlierRemove

with open("/media/root/6701ae9d-6612-4271-8d50-522d7f72528b/zht/Recursive_network_pytorch/data.pkl", "rb") as f:
    data = pickle.load(f)

outlier_remover = GrubbsOutlierRemove()
vols = data["simi"]["img_raw"]
raw_vol = np.zeros_like(vols[0], dtype=np.float32)
pts = []

for x in tqdm(range(vols[0].shape[0])):
    for y in range(vols[0].shape[1]):
        for z in range(vols[0].shape[2]):
            datas = outlier_remover.remove_outlier([i[x, y, z] for i in vols])
            pts.append(len(datas))
            raw_vol[x, y, z] = sum(datas) / len(datas)

In [ ]:
from collections import Counter
counter = Counter(pts)
print(counter)